## Music mapping flow 1
1. get user spotify ID
    - with user spotify ID, retrieve user spotify content list;
        - tracklist
        - audio qualities for respective song
        - lyrics for respective song
    - Backend done. Frontend needed for external use
    - [Next feature](https://towardsdatascience.com/get-your-spotify-streaming-history-with-python-d5a208bbcbd3): analysis of user streaming history and compare with playlist patterns
2. run existing Big5 text analysis to prepare collected data for review;
    - genres over time
    - MeyersBriggs v1 ft. Tufts buckets
3. return two interactive visualizations;
    - user music tastes over time
    - user music moods over time

### Step 1: Get user info

In [1]:
# testing on another user...

# variable setup
from spotify import *
from text_miner import *
songs = []
username = 12178525311 # --> JowMao ()
helper = spotifyApi()
txtMiner = textMiner()
sp = helper.sp
audio_df = pd.DataFrame()

# retrieve user playlists
playlists = helper.get_user_playlists(username=username, sp=sp)
for playlist in tqdm(playlists):
    
    # retrieve playlist songs
    tracklist = helper.get_playlist_content(username=username, playlist=playlist, sp=sp, csv=False)
    
    # retrieve playlist audio features
    tmp_df = helper.get_playlist_audio_features(username=username, playlist=playlist, sp=sp, csv=False)
    audio_df = pd.concat([audio_df, tmp_df], axis=0)
    
    # retrieve song lyrics (searching genius api)
    for track in tracklist:
        track['genius_url'] = None
        track_name = track["track"]["name"]
        track_artist = track["track"]["artists"][0]["name"]
        genius_song_info = txtMiner.search_genius_song(track_name=track_name, track_artist=track_artist, debug=False)
        if genius_song_info:
            track["track"]["genius_info"] = genius_song_info
        else:
            track["track"]["genius_info"] = {
                "result": {
                    "url": "https://genius.com/" + track_name.replace(" ", "-") + "-" + track_artist.replace(" ", "-") + "-" + "lyrics"
                }
            }
        track["track"]["lyrics"] = txtMiner.scrape_genius_lyrics(artistname=track_artist, songname=track_name, url=track["track"]["genius_info"]["result"]["url"])
    
    # save collected data for tracklist
    songs.extend(tracklist)
    
print(
    f"\nExtracted {len(songs)} songs from {len(playlists)} playlists from your Spotify Account.\nScraped lyrics for {len([s for s in songs if s['track']['lyrics'] != ''])} of the {len(songs)} songs from genius.com."
)

# savings objects here just in case...
txtMiner.save_df(filename=f"User_{username}_audioFeautures_", df=audio_df)
txtMiner.save_df(filename=f"User_{username}_songsNlyrics_", df=pd.DataFrame([songs]))

# success! 
# result of running this cell: 
# { 
#  'songs': list of songs in your playlists. Available lyrics from genius.com included, 
#  'audio_df': pandas dataframe containing the spotify-provided audio features present in each song in songs
# }

# NOTE: Connecting this to a user-interface will take a little tuning...could be hosted on AWS....on local server...

Name: b'tide pools ', Number of songs: 4, Playlist ID: 7HCAhke3De2jsnXb4ZFXaf 
Name: b'Unbeknownst Bangers', Number of songs: 27, Playlist ID: 3E684Qjlabk2LmcJjZ2zZQ 
Name: b'sampleLove', Number of songs: 116, Playlist ID: 0DYobfEIwMJlvVBymPoPCf 
Name: b'Costa Rica ', Number of songs: 0, Playlist ID: 46CQNWsKAfp3O2dD12aGV6 
Name: b'purpleParrots pt.3', Number of songs: 34, Playlist ID: 01RLihOalQPRbNAJdCiwOn 
Name: b'Study \xf0\x9f\x93\x9a', Number of songs: 53, Playlist ID: 5c68TrgrsR1COt97GYiHTm 
Name: b'yeye', Number of songs: 12, Playlist ID: 3YUa8BAtVbKmStMYcUnF5z 
Name: b"I'm Just Snacking", Number of songs: 23, Playlist ID: 7f7xecW1muGd5KamjC3EEM 
Name: b'la flame', Number of songs: 472, Playlist ID: 0i3CkxalKp9RRW4jf63PU5 
Name: b'Fashion Killa', Number of songs: 372, Playlist ID: 6k96myNgO28Lt9lZUPqdiU 
Name: b'vamos', Number of songs: 8, Playlist ID: 6pS0A7SrSkuxPSZnsaHd54 
Name: b'View From A Blue Moon', Number of songs: 13, Playlist ID: 0cOa2UTqNsjWO5AJswRZRA 
Name: b'Earga

100%|████████████████████████████████████████████████████████████████████████| 14/14 [32:10<00:00, 137.92s/it]



Extracted 1333 songs from 14 playlists from your Spotify Account.
Scraped lyrics for 983 of the 1333 songs from genius.com.


### Step 2: Data preparation

##### Step 2a: Audio preparation

In [6]:
# get latest set of audioFeatures (moods over time)
# data_path=txtMiner.data_path
# allfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]
# audio_dfs = [filename for filename in allfiles if "audioFeatures" in filename]
# audio_dfs.sort()
# audio_df = pd.read_csv(data_path + audio_dfs[-1])
audio_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1103 entries, 0 to 56
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   energy            1103 non-null   float64
 1   liveness          1103 non-null   float64
 2   tempo             1103 non-null   float64
 3   speechiness       1103 non-null   float64
 4   acousticness      1103 non-null   float64
 5   instrumentalness  1103 non-null   float64
 6   time_signature    1103 non-null   object 
 7   danceability      1103 non-null   float64
 8   key               1103 non-null   object 
 9   duration_ms       1103 non-null   object 
 10  loudness          1103 non-null   float64
 11  valence           1103 non-null   float64
 12  mode              1103 non-null   object 
 13  type              1103 non-null   object 
 14  uri               1103 non-null   object 
 15  playlist_id       1103 non-null   object 
dtypes: float64(9), object(7)
memory usage: 146.5

In [7]:
# TODO: Is cleaning necessary for these new datasets?

NOTES: 
    
    This is all bad. The output from step 1 should be a single denormalized dataset. Any alternative will result in disaster.
    

##### Step 2b: Song / Lyrics preparation (genres over time)